In [ ]:
# Set the needed context, create a hive database to import all the tables from the mysql database
from pyspark import SparkContext,SparkConf
import os
conf=SparkConf().setMaster("yarn-client").setAppName("pyspark")
sc=SparkContext(conf=conf)
from pyspark.sql import HiveContext
sqlContext=HiveContext(sc)
sqlContext.sql('create database if not exists problem6')
os.system("sqoop import-all-tables --connect jdbc:mysql://quickstart.cloudera:3306/retail_db --username retail_dba --password cloudera \
--warehouse-dir /user/hive/warehouse/problem6")




In [6]:
#Rank products within department by price and order by department ascending and rank descending
sqlContext.sql('select   d.department_id,   p.product_id,   p.product_name,  p.product_price,  rank() over (partition by d.department_id order by p.product_price) as product_price_rank,   dense_rank() over (partition by d.department_id order by p.product_price) as product_dense_price_rank   from products p   inner join categories c on c.category_id = p.product_category_id  inner join departments d on c.category_department_id = d.department_id  order by d.department_id, product_price_rank desc, product_dense_price_rank').show()


+-------------+----------+--------------------+-------------+------------------+------------------------+
|department_id|product_id|        product_name|product_price|product_price_rank|product_dense_price_rank|
+-------------+----------+--------------------+-------------+------------------+------------------------+
|            2|        66|  SOLE F85 Treadmill|      1799.99|               168|                      56|
|            2|        60| SOLE E25 Elliptical|       999.99|               167|                      55|
|            2|        74|Goaliath 54" In-G...|       499.99|               166|                      54|
|            2|       117|YETI Tundra 65 Ch...|       399.99|               164|                      53|
|            2|       162|YETI Tundra 65 Ch...|       399.99|               164|                      53|
|            2|        71|Diamondback Adult...|       349.98|               163|                      52|
|            2|       127|Stiga Master Seri...

In [7]:
prodRankByPrice=sqlContext.sql('select   d.department_id,   p.product_id,   p.product_name,  p.product_price,  rank() over (partition by d.department_id order by p.product_price) as product_price_rank,   dense_rank() over (partition by d.department_id order by p.product_price) as product_dense_price_rank   from products p   inner join categories c on c.category_id = p.product_category_id  inner join departments d on c.category_department_id = d.department_id  order by d.department_id, product_price_rank desc, product_dense_price_rank')

In [20]:
#find top 10 customers with most unique product purchases. if more than one customer has the same number of product purchases then the customer with the lowest customer_id will take precedence
top10CustByProdCnt=sqlContext.sql('select c.customer_id, count(distinct(oi.order_item_product_id)) unique_prod_count from customers c join orders o on c.customer_id=o.order_customer_id join order_items oi on o.order_id=oi.order_item_id group by c.customer_id order by count(distinct(oi.order_item_product_id)) desc, c.customer_id asc limit 10')

In [22]:
#Create Hive tables to capture the results
prodRankByPrice.registerTempTable('prodRankByPrice')
top10CustByProdCnt.registerTempTable('top10CustByProdCnt')


In [23]:
sqlContext.sql('desc prodRankByPrice').show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|       department_id|      int|       |
|          product_id|      int|       |
|        product_name|   string|       |
|       product_price|   double|       |
|  product_price_rank|      int|       |
|product_dense_pri...|      int|       |
+--------------------+---------+-------+



In [24]:
sqlContext.sql('desc top10CustByProdCnt').show()

+-----------------+---------+-------+
|         col_name|data_type|comment|
+-----------------+---------+-------+
|      customer_id|      int|       |
|unique_prod_count|   bigint|       |
+-----------------+---------+-------+



In [26]:
#Now create Hive tables using the temp tables from the dataframes
sqlContext.sql('create table if not exists default.top10Cust as select *  from top10CustByProdCnt')

sqlContext.sql('create table if not exists default.prodPriceRanked as select *  from prodRankByPrice')

DataFrame[]